# Problem logarytmu dyskretnego

Niech $(G,\circ)$ będzie grupą z działaniem $\circ$ i elementem neutralnym $1_G$. Wtedy dla dowolnego elementu $a\in G$ i $k\in\mathbb{Z}$ definiujemy *potęgę* $$a^k =\left\{\begin{array}{cc}
\underbrace{a\circ a\circ \ldots \circ a}_{k}&\text{ dla }k>0\\
1_G&\text{ dla }k=0\\
\underbrace{a^{-1}\circ a^{-1}\circ \ldots \circ a^{-1}}_{k}&\text{ dla }k<0
\end{array}\right.$$
gdzie $a^{-1}$ jest elementem odwrotnym do $a$.

Dla $a,b\in G$, $b\neq 1_G$, **logarytmem dyskretnym** $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a$.

## Logarytm dyskretny w $\mathbb{Z}_n$

W przypadku pierścienia $\mathbb{Z}_n$ logarytmem dyskretnym $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a\mod n$, o ile w ogóle istnieje.

Specyficzną sytuacją w teorii liczb jest gdy $n=p$ jest liczbą pierwszą a $q$ jest **pierwiastkiem pierwotnym** $\mod p$. Wtedy:
- potęgi $q^k\mod p$ generują cały zbiór $[1,p-1]$, tzn. $q$ jest **generatorem grupy multiplikatywnej** rzędu $p-1$
- logarytm dyskretny $\log_q a$ istnieje dla każdego niezerowego elementu $a\in \mathbb{Z}_p$


# Wymiana klucza typu Diffie-Hellman

Alice i Bob uzgadniają klucz publiczny będący liczbą pierwszą $p$ oraz $q$ - pierwiastkiem pierwotnym mod $p$.
- sekret Alice: liczba całkowita $n\in \mathbb{Z}_p\setminus\{0\}$
- sekret Boba: liczba całkowita $m\in \mathbb{Z}_p\setminus\{0\}$
- Alice generuje $x=q^n\mod p$ i wysyła do Boba
- Bob generuje $y=q^m\mod p$ i wysyła Alice
- Alice oblicza klucz $k=y^n\mod p$
- Bob oblicza klucz $k=x^m\mod p$


## Zadanie 1.

Zaimplementuj powyższy algorytm wymiany klucza. Dobierz parametry $p$ i $q$ tak, żeby znając $x$, $y$, $p$ i $q$ nie dało się odtworzyć sekretów algorytmem z zadania 1.

In [2]:
import random
import math

In [3]:
def prime_naive(n):
    if n < 2:
        return False

    m = math.isqrt(n)
    for i in range(2, m + 1):
        if n % i == 0:
            return False

    return True

In [4]:
def prime_miller_rabin(n, k=1):
    if n < 2:
        return False
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False

    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

In [5]:
primes_less_than_2000 = [x for x in range(2, 2000) if prime_naive(x)]

def generatePrime(keysize):
    result = False

    while not result:
        result2 = True
        p = random.getrandbits(keysize)
        p |= (1 << 0)
        p |= (1 << (keysize - 1))

        for prime in primes_less_than_2000:
            if p % prime == 0 and p != prime:
                result2 = False
                break
        
        if not result2: 
            continue
        
        result = prime_miller_rabin(p, 5)
    return p

In [6]:
def factorize(n):
    factors = set()
    d = 2
    while d * d <= n:
        while n % d == 0:
            factors.add(d)
            n //= d
        d += 1
    if n > 1:
        factors.add(n)
    return factors

def primitive_root(p):
    phi = p - 1
    factors = factorize(phi)

    for g in range(2, p):
        ok = True
        for q in factors:
            if pow(g, phi // q, p) == 1:
                ok = False
                break
        if ok:
            return g

In [8]:
p = generatePrime(16)
q = primitive_root(p)

print(f"Publiczne parametry: p={p}, q={q}")

n = random.randrange(1, p)  # sekret Alice
m = random.randrange(1, p)  # sekret Boba

print(f"Sekret Alice: n={n}")
print(f"Sekret Boba:  m={m}")

# Alice
x = pow(q, n, p)

# Bob
y = pow(q, m, p)

print(f"Alice wysyła x = {x}")
print(f"Bob wysyła y = {y}")

key_Alice = pow(y, n, p)
key_Bob = pow(x, m, p)

print(f"Klucz Alice: {key_Alice}")
print(f"Klucz Bob:   {key_Bob}")

Publiczne parametry: p=39887, q=7
Sekret Alice: n=23474
Sekret Boba:  m=22785
Alice wysyła x = 5137
Bob wysyła y = 36164
Klucz Alice: 18356
Klucz Bob:   18356


## Algorytm baby-step giant-step

Jeden z najprostszych (poza metodą naiwną) algorytmów poszukiwania logarytmu dyskretnego w grupach cyklicznych.

Niech $p$ będzie liczbą pierwszą oraz niech $q$ będzie pierwiastkiem pierwotnym modulo $p$. Dla niezerowego $a\in\mathbb{Z}_p$ szukamy liczby $k\in\mathbb{Z}$ takiej, że $q^k=a\mod p$

### Krok 1.
- $m=\lceil\sqrt{p-1}\rceil$
- tworzymy pomocniczą tablicę potęg: dla wszystkich $i\in [0,m)$ obliczamy parę $(i,q^i)$
- obliczamy $r=(q^{-1})^m$
### Krok 2.
- $b=a$
- dla wszystkich $j\in [0,m)$:
    - sprawdzamy, czy para $(i,b)$ jest elementem tablicy potęg dla pewnego $i$
    - jeżeli tak, to $k=jm+i$ i kończymy algorytm
    - jeżeli nie, to $b=br$ i kontynuujemy pętlę

## Zadanie 2.

Zaimplementować algorytm baby-step giant-step. Przetestować dla podanych danych testujących.

```Dane testujące:
p = 7
q = 3
a = 4

m = 3
tablica_testowa = [1,3,2]
r = 6
k = 4 (j = 1, i = 1)
```

```
p = 29
q = 8
a = 10

m = 6
tablica_testowa = [1,8,6,19,7,27]
r = 9
k = 17 (j = 2, i = 5)
```

```
p = 113
q = 76
a = 84

m = 11
tablica_testowa = [1,76,13,84,56,75,50,71,85,19,88]
r = 70
k = 3 (j = 0, i = 3)
```

In [23]:
def baby_step_giant_step(p, q, a):
    """
    Zwraca k takie, że q^k = a (mod p), o ile istnieje.
    """
    m = math.isqrt(p - 1) + 1

    baby = {}
    current = 1
    for i in range(m):
        baby[current] = i
        current = (current * q) % p

    q_inv = pow(q, p - 2, p)
    r = pow(q_inv, m, p)

    b = a
    for j in range(m):
        if b in baby:
            i = baby[b]
            return j * m + i

        b = (b * r) % p

    return None

In [24]:
tests = [
    (7, 3, 4, 4),     # k = 4
    (29, 8, 10, 17),  # k = 17
    (113, 76, 84, 3)  # k = 3
]

for p, q, a, expected in tests:
    result = baby_step_giant_step(p, q, a)
    print(f"p={p}, q={q}, a={a} -> k={result}, expected={expected}")

p=7, q=3, a=4 -> k=4, expected=4
p=29, q=8, a=10 -> k=17, expected=17
p=113, q=76, a=84 -> k=3, expected=3
